In [5]:
from BTools import *


In [6]:
mika = pairs()
for i in mika.get("BTC"):
    print(i)

[<BTools.pair object at 0x7f6d7c336970>, <BTools.pair object at 0x7f6d7c34c0a0>, <BTools.pair object at 0x7f6d7c34c9d0>, <BTools.pair object at 0x7f6d7c3426d0>, <BTools.pair object at 0x7f6d7c342ee0>, <BTools.pair object at 0x7f6d7c342f70>, <BTools.pair object at 0x7f6d7c1194f0>, <BTools.pair object at 0x7f6d7c1194c0>, <BTools.pair object at 0x7f6d7c119610>, <BTools.pair object at 0x7f6d7c119940>]
